Answers to within notebook tasks
=====

Note: this notebook does not contain the answers to the extra tasks

In [ ]:
# Example for getting the lengths of each array in notebook 2b_training

len_seq_train = [len(ar) for ar in train]
len_seq_test = [len(ar) for ar in test]

In [ ]:
# Example for creating the random emission probability array in notebook 2b_training

em_prob = np.array([[np.random.random() if y == 'rand' else y for y in x] for x in em_prob_o], dtype = np.float64)
em_prob = np.array([[y / sum(x) for y in x] for x in em_prob], dtype = np.float64)

In [ ]:
# Example of how to complete the final task in notebook 2b_training

observables = ['immobile', 'micro', 'walking']
hidden_states = ['deep sleep', 'light sleep', 'quiet awake', 'active awake']

t_prob_o = np.array([['rand', 'rand', 'rand', 0.0],
                    ['rand', 'rand', 'rand', 0.0],
                    [0.0, 'rand', 'rand', 'rand'],
                    [0.0, 0.0, 'rand', 'rand']])


em_prob_o =  np.array([[1.0, 0.0, 0.0],
                    [1.0, 0.0, 0.0],
                    ['rand', 'rand', 'rand'],
                    ['rand', 'rand', 'rand']])

model = CategoricalHMM(n_components = len(hidden_states),  n_features = len(observables), n_iter = 50, tol = 5, params = 'ste', init_params = 's', verbose = True) 

save_path = '...\HMM_tutorial\data\4_state_hmm.pkl'

iterations = 10 
for i in range(iterations):

    t_prob = np.array([[np.random.random() if y == 'rand' else y for y in x] for x in t_prob_o], dtype = np.float64)
    t_prob = np.array([[y / sum(x) for y in x] for x in t_prob], dtype = np.float64)
    model.transmat_ = t_prob

    em_prob = np.array([[np.random.random() if y == 'rand' else y for y in x] for x in em_prob_o], dtype = np.float64)
    em_prob = np.array([[y / sum(x) for y in x] for x in em_prob], dtype = np.float64)
    model.emissionprob_ = em_prob

    hmm = model.fit(seq_train, len_seq_train)
    print("True Convergence:" + str(hmm.monitor_.history[-1] - hmm.monitor_.history[-2] < hmm.monitor_.tol)) # If it converges rather than max iterations print True
    print("Final log liklihood score:" + str(hmm.score(seq_train, len_seq_train))) # Print the final log liklihood score

    score = hmm.score(seq_test, len_seq_test) 

    if best_score is None or best_score < score:
        best_score = score
        best_model = hmm
        print('New Matrix: \n')
        print(f'Transition matrix: \n {hmm.transmat_} \n')
        print(f'Emission probabilities: \n {hmm.emissionprob_}')

with open(save_path, "wb") as file: pickle.dump(best_model, file)
hmm_display(hmm, hidden_states, observables)